# KNeighbors Classification

In [1]:
from sklearn.datasets import  load_iris

In [2]:
iris = load_iris()
data =  iris.data
target = iris.target

In [3]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3)

In [9]:
from sklearn.neighbors import KNeighborsClassifier

In [10]:
model = KNeighborsClassifier()

In [13]:
model.fit(X_train, y_train)

KNeighborsClassifier()

In [15]:
predictions = model.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score

In [19]:
accuracy = accuracy_score(y_test, predictions)

In [21]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9777777777777777


# Hyperparameter Tuning using GridSearchCV

In [24]:
from sklearn.model_selection import GridSearchCV

In [26]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

In [28]:
grid_search = GridSearchCV(model, param_grid, cv=2)

In [30]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_neighbors': [3, 5, 7, 9, 11],
                         'weights': ['uniform', 'distance']})

In [32]:
grid_search.best_params_

{'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}

In [34]:
best_k = grid_search.best_params_["n_neighbors"]

In [36]:
best_model = KNeighborsClassifier(n_neighbors = best_k)

In [38]:
best_model.fit(X_train, y_train)

KNeighborsClassifier()

In [40]:
predictions = best_model.predict(X_test)

In [42]:
accuracy = accuracy_score(y_test, predictions)

In [44]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9777777777777777


# Hyperparameter Tuning using Optuna

In [47]:
import optuna

In [49]:
# Define objective function for Optuna
def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15, step=2)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])

    model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    Accuracy = accuracy_score(y_test, y_pred)
    return Accuracy

In [51]:
# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-03-01 14:43:55,285] A new study created in memory with name: no-name-53c1969a-fa50-4715-9ea5-39933faa14c3
[I 2025-03-01 14:43:55,295] Trial 0 finished with value: 0.9555555555555556 and parameters: {'n_neighbors': 9, 'weights': 'distance', 'metric': 'manhattan'}. Best is trial 0 with value: 0.9555555555555556.
[I 2025-03-01 14:43:55,304] Trial 1 finished with value: 0.9777777777777777 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 1 with value: 0.9777777777777777.
[I 2025-03-01 14:43:55,319] Trial 2 finished with value: 0.9555555555555556 and parameters: {'n_neighbors': 11, 'weights': 'uniform', 'metric': 'euclidean'}. Best is trial 1 with value: 0.9777777777777777.
[I 2025-03-01 14:43:55,335] Trial 3 finished with value: 0.9555555555555556 and parameters: {'n_neighbors': 11, 'weights': 'uniform', 'metric': 'euclidean'}. Best is trial 1 with value: 0.9777777777777777.
[I 2025-03-01 14:43:55,342] Trial 4 finished with value: 0.9555

In [53]:
study.best_params

{'n_neighbors': 5, 'weights': 'uniform', 'metric': 'manhattan'}

In [55]:
study.best_value

0.9777777777777777

In [57]:
best_params = study.best_params

In [59]:
best_model1 = KNeighborsClassifier(**best_params)

In [61]:
best_model1.fit(X_train, y_train)

KNeighborsClassifier(metric='manhattan')

In [63]:
predictions = best_model1.predict(X_test)

In [65]:
accuracy = accuracy_score(y_test, predictions)

In [67]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9777777777777777
